# Pronóstico adaptativo

## Descripción del problema real

Los pronósticos de los precios de la electricidad en mercados liberalizados son un insumo fundamental para la toma de decisiones dentro de las organizaciones. Fundamentalmente, los pronosticos de corto plazo son utilizados en decisiones de carácter operativo. En el caso abordado, es necesario contar con los pronósticos para definir la politica de operación de una empresa del sector eléctrico.

## Descripción del problema en términos de los datos

La carpeta `datos/precios/` contiene los precios historicos horarios de la electricidad en la Bolsa de Energía del mercado eléctrico colombiano, publicados por el operador del mercado. Se desean obtener pronósticos para el precio promedio diario para los próximos siete (7) días a la fecha actual.

## Aproximaciones posibles

En este caso, se desea evaluar la capacidad de pronóstico de un ADALINE con aprendizaje en línea.

## Requerimientos

Usted debe:

* Procesar los datos históricos para conformar el conjunto de datos. Los archivos de Excel no pueden ser modificados y actualizados directamente por el operador del sistema. Su código debe leer los archivos y crear la serie de precios promedio diarios de la electricidad.


* Determinar si el modelo debe pronosticar los precios promedios sin ninguna transformación, o si la inclusión de una transformación (logaritmo natural, raíz cúbica, raíz cuadrada, etc) resulta en un incremento de la precisión del pronóstico.


* Generar el pronóstico para los precios de los próximos siete días.


* Preparar el código para que el modelo sea entrenado usando el gradiente y el momentum.


* Determinar cuál es el número óptimo de retardos (observaciones) que el modelo debe considerar hacia atrás para producir el pronóstico.


* Determinar los valores óptimos de la tasa de aprendizaje y el momentum.


## Librerías de uso

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

## Preprocesamiento de los datos
Para el preprocesamiento de los datos fue necesario añadir varias excepciones para la lectura de datos, ya que el formato de 
estos no era consistente en su totalidad

In [ ]:
file_path = './datos/precios/Precio_Bolsa_Nacional_($kwh)_'

complete_df = None
for year in range(1995,2019):
    extension = 'xlsx'
    if (year >= 2016):
        extension = 'xls'
    filestring = file_path + str(year) +'.'+ extension
    skiprows=3
    if (year >= 2000):
        skiprows = 2
    df = pd.read_excel(filestring,skiprows=skiprows)
    means = []
    df = df.dropna(axis='index', thresh=10)
    df = df.dropna(axis='columns', how='all')
    for index, row in df.iterrows():
        s = 0
        c = 0
        for i in range(0,24):
            value = row[str(i)]
            if (not pd.isnull(value)):
                s += float(value)
                c += 1
        prom = s/c
        means.append(prom)
    df['Mean'] = means
    if (year == 1995):
        complete_df = df
    else:
        complete_df = complete_df.append(df)
parsed_dates = []
for d in complete_df['Fecha'].values:
    if (isinstance(d,str)):
        parsed_dates.append(dt.datetime.strptime(d,"%Y-%m-%d"))
    elif (isinstance(d,dt.datetime)):
        parsed_dates.append(d)
    else:
        print(d)
        print(type(d))
complete_df['Parse Date'] = pd.to_datetime(parsed_dates)

complete_df['Timestamp'] = [d.timestamp() for d in parsed_dates]
complete_df.describe()

### Vista previa de los datos originales

In [ ]:
complete_df.head()


## Gráfica de la tendencia promedio de los precios

Se realiza una gráfica para observar el comportamiento de los precios a lo largo de los años, se identifica el gran pico en la crisis energética de 2016

In [ ]:
complete_df.sort_values(by='Parse Date')
complete_df.drop_duplicates(subset="Parse Date", keep="first", inplace=True)
plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('COP')
plt.plot(complete_df['Parse Date'], complete_df['Mean'], color='red')
plt.show()

## IPC 
Es necesario normalizar los datos multiplicandolos por el IPC histórico y así poder descartar la inflación como variable de cambio, para esto se hará uso del archivo de excel 'ipc_historico.xlsx, que también se encuentra en la carpeta de datos, este archivo se obtuvo de la página web del <a href="https://www.banrep.gov.co/es/estadisticas/indice-precios-consumidor-ipc" target="_blank">Banco de la República</a>  

In [ ]:
ipc_df = pd.read_excel('./datos/precios/series_ipc_historico.xlsx', skiprows=12, parse_dates=True)
ipc_df = ipc_df.dropna(axis='index', thresh=2)
ipc_df = ipc_df.dropna(axis='columns', how='all')
ipc_df['Parse Date'] = [dt.datetime.strptime(str(d),"%Y%m") for d in ipc_df['Año(aaaa)-Mes(mm)'].tolist()]
ipc_df.sort_values(by='Parse Date', inplace=True)
ipc_df.tail()



In [ ]:
#Obtenemos el IPC base
mask = (ipc_df['Índice'] == 100)
base_ipc_df = ipc_df.loc[mask]
base_ipc_df.head()

In [ ]:
base_ipc = base_ipc_df['Índice'].tolist()[0]
print(base_ipc)

In [ ]:
normalized_means = []
for m,d in zip(complete_df['Mean'].tolist(), complete_df['Timestamp'].tolist()):
    date = dt.datetime.utcfromtimestamp(d)
    mask = (ipc_df['Parse Date'].dt.year == date.year) & (ipc_df['Parse Date'].dt.month == date.month)
    ipc = ipc_df.loc[mask]['Índice'].values[0]
    normalized_price = m * (base_ipc/ipc)
    normalized_means.append(normalized_price)

complete_df['Precio Normalizado'] = normalized_means    

complete_df.tail()

## Comparación de Precio vs Precio Normalizado

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('COP')
plt.plot(complete_df['Parse Date'], complete_df['Mean'], label="Precio", color='red')
plt.plot(complete_df['Parse Date'], complete_df['Precio Normalizado'], label="Precio Normalizado", color='black')
plt.legend()
plt.show()

# Transformación de los Datos
Finalizado el proceso de preprocesamiento, se conformará un nuevo dataframe con los datos estrictamente necesarios, y una a una se irán agregando las transformaciones a los precios normalizados. Las transformaciones a usar serán:
<ul>
    <li>Logaritmo natural</li>
    <li>Raíz cuadrada</li>
    <li>Raíz cúbica</li>
</ul>

In [ ]:
def transform(values,trans, inverse = False):
    if (not inverse):
        if (trans == "Log_Natural"):
            return np.log(values)
        elif (trans == "Raiz_Cuadrada"):
            return np.sqrt(values)
        elif (trans == "Raiz_Cubica"):
            return np.cbrt(values)
    else:
        if (trans == "Log_Natural"):
            return np.exp(values)
        elif (trans == "Raiz_Cuadrada"):
            return np.power(values,2)
        elif (trans == "Raiz_Cubica"):
            return np.power(values,4)

##Nuevo dataframe solo con los datos requeridos
df_dict = {
    "Fecha": complete_df['Parse Date'],
    "Timestamp": complete_df['Timestamp'],
    "Precio_Promedio": complete_df['Mean'],
    "Precio_Normalizado": complete_df['Precio Normalizado']
}
df = pd.DataFrame(df_dict)
transformations = ['Log_Natural','Raiz_Cuadrada', 'Raiz_Cubica']
for t in transformations:
    ##Le agregamos las distintas transformaciones al precio normalizado
    df[t] = transform(df['Precio_Normalizado'],t)
df.describe()

## Gráfica comparativa de cada una de las transformaciones

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('Valor')
#plt.plot(df['Fecha'], df['Precio_Normalizado'], label="Precio Normalizado", color='black')
plt.plot(df['Fecha'], df['Log_Natural'], label="Log Natural", color='red')
plt.plot(df['Fecha'], df['Raiz_Cuadrada'], label="Raíz Cuadrada", color='green')
plt.plot(df['Fecha'], df['Raiz_Cubica'], label="Raíz Cúbica", color='blue')

plt.legend()
plt.show()

## Elección de la Transformación

Para elegir la transformación adecuada se realizará el modelo, se hará un pronóstico y se seleccionará aquella transformación con menor error


### Implementación del Modelo

La implementación del modelo escogida es la propuesta por el profesor en el siguiente <a href="https://jdvelasq.github.io/courses/notebooks/tensorflow/adaline/1-01-adalines.html" target="_blank">enlace</a>

In [ ]:
class Adaline:
    def __init__(self,
                 learning_rate=0.001, # tasa de aprendizaje
                 momentum=0,      # Momentum               
                 max_epochs=100,      # número máximo de iteraciones sobre el set de datos
                 shuffle=False,       # mezcla patrones para aprendizaje online
                 random_state=None,   #
                 warm_start=False):   #
        
        self.momentum = momentum
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.shuffle = shuffle
        self.random_state = random_state
        self.warm_start = warm_start
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):

        if not isinstance(X, np.ndarray):
            X = np.array(X)
        if not isinstance(y, np.ndarray):
            d = np.array(y)
        else:
            d = y.copy()

        if self.random_state is not None:
            np.random.RandomState(self.random_state)

        if self.coef_ is None or self.warm_start is False:
            self.coef_ = np.random.uniform(-1, 1, X.shape[1])

        if self.intercept_ is None  or self.warm_start is False:
            self.intercept_ = np.random.uniform(-1, 1, 1)

        errors2 = []
        forecasts = []

        for epoch in range(self.max_epochs):

            if self.shuffle is True:
                n = list(range(X.shape[0]))
                np.random.shuffle(n) #Desordena una lista de indices
                X = X[n,:] 
                d = d[n]

            for i in range(X.shape[0]):
                u = np.dot(X[i,:], self.coef_) + self.intercept_ 
                e = (d[i] - u)[0]
                ## Se agrega el momentum al coeficiente e intercepto
                self.coef_ += 2 * self.learning_rate * e * X[i,:] + self.coef_ * self.momentum
                self.intercept_ += 2 * self.learning_rate * e + self.intercept_*self.momentum
                errors2.append(e**2)
                forecasts.append(u)

        return errors2, forecasts

    def predict(self, X):
        if not isinstance(X, np.ndarray):
            X = np.array(X)
        u = np.dot(X, self.coef_) + self.intercept_
        return u

In [ ]:
transformations = ['Log_Natural', 'Raiz_Cuadrada', 'Raiz_Cubica']
L = 7
scalers = dict() ## Scalers para cada transformación
desireds_d = dict() ## Deseados 
forecasts_d = dict() ## Pronóstico
errors2_d = dict() ## errores

## Realizar el pronóstico para cada transformación
for trans in transformations:
    scaler = MinMaxScaler() ##Scaler
    d = np.asarray(df[trans]).reshape(-1,1)
    scaler.fit(d)
    scaled_d = scaler.transform(d).ravel() ##Usar los valores escalados entre 0 y 1 
    scalers[trans] = scaler
    desireds_d[trans] = d
    X = []
    for t in range(L, len(d)):
        X.append(scaled_d[t-L:t].copy())
        
    adaline = Adaline(
    learning_rate=0.001, 
    max_epochs=1,        
    shuffle=False,       
    random_state=123,    
    warm_start=False)    
    
    errors2, forecasts = adaline.fit(X,scaled_d[L:])
    forecasts_d[trans] = scaler.inverse_transform(np.asarray(forecasts).reshape(-1,1)).ravel() ##Regresar a la escala
    errors2_d[trans] = errors2



In [ ]:
plt.figure(figsize=(20,15)) 
for i in range(len(transformations)):
    trans = transformations[i]
    plt.subplot(3,1,i+1)   
    plt.xlabel('Año')
    plt.ylabel('Valor')
    plt.plot(desireds_d[trans], label=trans, color='black')
    plt.plot(range(L, len(desireds_d[trans])),forecasts_d[trans], label="Pronóstico", color='red')
    plt.legend()


Se observa que todas las transformaciones tienen un comportamiento similar, un poco errático por la alta variación de los datos. Entre estas la que más destaca es la transformación de Log_Natural, de igual manera se realizará el análisis modificando las tasa de aprendizaje y otros valores.

In [ ]:
## Aplicando la transformación inversa
forecast_inv = dict()
for t in transformations:
    forecast_inv[t] = transform(forecasts_d[t],t,True)

plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('Valor')
first_date = dt.datetime.utcfromtimestamp(df['Timestamp'].tolist()[0])
date_thresh = (df['Fecha'] >= first_date + dt.timedelta(L-1))
dates_df = df.loc[date_thresh]['Fecha']
plt.plot(df['Fecha'],df['Precio_Normalizado'], label="Precio_Normalizado", color='black')
plt.plot(dates_df,forecast_inv["Log_Natural"], label="Pronósitco Log", color='red')
plt.plot(dates_df,forecast_inv["Raiz_Cubica"], label="Pronóstico Raiz3", color='blue')

plt.legend()
plt.show()

Realizando la comparación se puede notar que las transformaciones tienen un comportamiento similar, ahora se procederá a evaluar el error.

## Separación de Datos (Predicción y Entrenamiento)

In [ ]:
models = ['Precio_Normalizado'] +transformations

aux_scaler = MinMaxScaler()
aux_scaler.fit(np.asarray(df['Precio_Normalizado']).reshape(-1,1))
scalers['Precio_Normalizado'] = aux_scaler ##Agregamos el scaler para el precio normalizado

n = df['Precio_Normalizado'].size 


train_df = pd.DataFrame(columns=models) ##Dataframe de entrenamiento

predict = df.iloc[(n - 7):]['Precio_Normalizado'] ##Datos a predecir
for model in models: 
    train_df[model] = df[model][:(n-7)]

Se guardan los resultados de los errores del modelo en un dataframe

In [ ]:
Ls= list(range(7, 14)) #Latencia
result_df = pd.DataFrame(columns = ["Modelo","E_Pronostico","E_Entrenamiento","L","Momentum","Tasa_Apr","Pronosticos"])

for model in models:
    d = np.asarray(train_df[model]).reshape(-1,1)
    scaler = scalers[model]
    d = scaler.transform(d).ravel() #Datos para usar en el modelo
    for L in Ls:
        for momentum in np.arange(0.00000, 0.0001,0.000025): #Se eligieron valores muy bajos del momentum porque variaba demasiado
            for learning_rate in np.arange(0.005, 0.1, 0.005):
                X = []
                P = []
                for t in range(L,len(d)):
                    X.append(d[t-L:t].copy())

                #Se entrena el modelo
                adaline = Adaline(
                    learning_rate=learning_rate, 
                    momentum = momentum,  
                    shuffle=False,       
                    random_state=123,    
                    warm_start=False)    
                errors2, forecasts = adaline.fit(X, d[L:])

                #Pronóstico
                for i in range(7):
                    u = adaline.predict(X[-1]) #Predicción proximo valor
                    next_X = np.append(X[-1][1:L], [u]) #Se agrega el nuevo pronóstico
                    X = np.concatenate( ( X, [next_X] ), axis=0) 
                    P.append(u[0]) #Almacena el pronóstico

                #Invertir la transformación de escalado    
                P = scaler.inverse_transform(np.asarray(P).reshape(-1,1)).ravel() 
                forecasts = scaler.inverse_transform(np.asarray(forecasts).reshape(-1,1)).ravel() 
                
                #Invertir la transformación aplicada
                if (model != 'Precio_Normalizado'):
                    P = transform(P,model,True)  
                    forecasts = transform(forecasts,model,True)

                e_p = np.sum(np.power(np.array(predict) - P, 2))   
                e = np.sum(errors2)
                #Guardar valores del pronóstico
                result_df = result_df.append({
                    "Modelo": model,
                    "E_Pronostico": e_p,
                    "E_Entrenamiento": e,
                    "L": L,
                    "Momentum": momentum,
                    "Tasa_Apr": learning_rate,
                    "Pronosticos_E":forecasts,
                    "Pronosticos": P,
                },ignore_index=True)
                
result_df.describe()

In [ ]:
best_predict = result_df[result_df.E_Pronostico == result_df.E_Pronostico.min()] ##Mejor pronóstico
best_train = result_df[result_df.E_Entrenamiento == result_df.E_Entrenamiento.min()] ##Mejor entrenamiento

best_predict.head() 

In [ ]:
best_train.head()

## Modelo seleccionado
Finalmente se selecciona el modelo con un menor error de entrenamiento para realizar la predicción de los próximos 7 días. De igual forma se grafica el modelo con el mejor pronóstico para contemplar el comportamiento del pronóstico realizado vs el real, donde se puede observar un comportamiento casi similar en cuestión de subida y caída del precio.

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('Valor')
plt.plot(predict.values, label="Datos", color='black', marker='o', linestyle='dashed',linewidth=2, markersize=12)
plt.plot(best_predict['Pronosticos'].values[0],label="Pronóstico", color='red', marker='o', linestyle='dashed',linewidth=2, markersize=12)

plt.legend()
plt.show()

### Comportamiento general del Modelo

Se gráfica el comportamiento general del modelo seleccionado

In [ ]:
#forecasts_p = best_predict['Pronosticos_E'].values[0]
forecasts_t = best_train['Pronosticos_E'].values[0]

plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('Valor')
plt.plot(train_df['Precio_Normalizado'].values, label="Precio Real", color = "black")
plt.plot(forecasts_t, label="Pronóstico", color="red")

plt.legend()
plt.show()

## Pronóstico de los 7 días con el modelo escogido

In [ ]:
##Se obtienen los datos del mejor pronóstico
transformation = best_train['Modelo'].values[0]
learning_rate = best_train['Tasa_Apr'].values[0]
momentum =  best_train['Momentum'].values[0]
L = best_train['L'].values[0]
epochs = best_train['Epochs'].values[0]

d = np.asarray(df[transformation]).reshape(-1,1)
scaler = scalers[transformation]
d = scaler.transform(d).ravel() ##Datos para usar en el modelo

X = []
P = []
for t in range(L,len(d)):
    X.append(d[t-L:t].copy())

adaline = Adaline(
    learning_rate=learning_rate,  
    momentum = momentum, 
    max_epochs=epochs,        
    shuffle=False,       
    random_state=123,    
    warm_start=False)    
errors2, forecasts = adaline.fit(X, d[L:]) 

##Pronósticar los próximos 7 días
for i in range(7):
    u = adaline.predict(X[-1]) 
    next_X = np.append(X[-1][1:L], [u]) 
    X = np.concatenate( ( X, [next_X] ), axis=0)
    forecasts.append(u[0]) 
    P.append(u[0])
    
forecasts = scaler.inverse_transform(np.asarray(forecasts).reshape(-1,1)).ravel()
forecasts = transform(forecasts,model,True)
P = scaler.inverse_transform(np.asarray(P).reshape(-1,1)).ravel()
P = transform(P,model,True)

    


### Comportamiento del Precio de la Energía para los próximos 7 días

In [ ]:
plt.figure(figsize=(20,5))
plt.title('Precio de la Energía')
plt.xlabel('Año')
plt.ylabel('Valor')
plt.plot(P, label="Pronóstico", color="red")

plt.legend()
plt.show()